In [1]:
from aind_codeocean_api.codeocean import CodeOceanClient, CodeOceanCredentials
from datetime import datetime
from pathlib import Path

import pandas as pd
import json

# 0. Connect to CodeOcean

In [2]:
domain = "https://codeocean.allenneuraldynamics.org/"
token = "cop_ZDcyZjY4MDBhMGNmNDVkNGI1MjdjZTI5ODVkODU3YWJlWmhxSVV5bDR4dWxDTTVoUXJqZGF6OWViUVJXMmwxMmYyMTdmZDhj"
co_client = CodeOceanClient(domain=domain, token=token)

# 1. Get Data Assest with id

In [3]:
# data asset
data_asset_id = "6314acc5-4275-4f4a-a183-07bf093e349f"
response = co_client.get_data_asset(data_asset_id=data_asset_id)
metadata = response.json()
metadata

{'created': 1692449614,
 'custom_metadata': {'data level': 'raw data',
  'experiment type': 'multiplane-ophys',
  'modality': 'Optical physiology',
  'subject id': '472271'},
 'description': '',
 'files': 2273,
 'id': '6314acc5-4275-4f4a-a183-07bf093e349f',
 'last_used': 0,
 'name': 'multiplane-ophys_472271_2019-10-07_05-46-00',
 'size': 195238734812,
 'sourceBucket': {'bucket': 'aind-ophys-data',
  'origin': 'aws',
  'prefix': 'multiplane-ophys_472271_2019-10-07_05-46-00'},
 'state': 'ready',
 'tags': ['multiplane-ophys', 'raw', '472271'],
 'type': 'dataset'}

# 2. Query all data assets

In [18]:
query = "multiplane"

response = co_client.search_all_data_assets(query=query)
mp = response.json()
results = mp['results']
print(f"Found {len(results)} results")
results[0:3]

# dump to json
# with open('results_from_mjd.json', 'w') as outfile:
#     json.dump(results, outfile)


Found 510 results


[{'created': 1692423326,
  'custom_metadata': {'data level': 'raw data',
   'experiment type': 'multiplane-ophys',
   'modality': 'Optical physiology',
   'subject id': '484408'},
  'description': '',
  'files': 5193,
  'id': '8a79a736-a892-4a87-b907-97a25975ef89',
  'last_used': 0,
  'name': 'multiplane-ophys_484408_2019-11-04_01-21-00',
  'size': 384001873508,
  'sourceBucket': {'bucket': 'aind-ophys-data',
   'origin': 'aws',
   'prefix': 'multiplane-ophys_484408_2019-11-04_01-21-00'},
  'state': 'ready',
  'tags': ['multiplane-ophys', 'raw', '484408'],
  'type': 'dataset'},
 {'created': 1692398549,
  'custom_metadata': {'data level': 'raw data',
   'experiment type': 'multiplane-ophys',
   'modality': 'Optical physiology',
   'subject id': '499478'},
  'description': '',
  'files': 1360,
  'id': 'ef94bc7c-d5cc-4970-a8ed-2656d0315d9b',
  'last_used': 0,
  'name': 'multiplane-ophys_499478_2020-02-21_01-10-00',
  'size': 343930657037,
  'sourceBucket': {'bucket': 'aind-ophys-data',
  

In [19]:
# filter by processed and raw data (1/10/2024 structure may change in future)

raw_data = []
processed_data = []
for result in results:
    try: 
        if result['custom_metadata']['data level'] == 'raw data':
            raw_data.append(result)
        else:
            # maybe custom metadata key will be added to processed in future
            processed_data.append(result)
    except KeyError:
        processed_data.append(result)

# print n of each
print(f"Found {len(raw_data)} raw data results")
print(f"Found {len(processed_data)} processed data results")

Found 384 raw data results
Found 126 processed data results


# 3. Matching LIMS date-time to CO date-time

1) Get data asset datetime
    + Get a date from the string in the data assest "name" key (e.g. multiplane-ophys_477052_2019-10-17_16-15-26)
    + convert to datetime object
2) Get ophys_session datetime (stored in json on lims)
    + Get ophys_session table from lims
    + For each row, open platform.json that stores correct datetime in the first imaging group key
    + add that datetime to the table
3) find matching datetimes



In [24]:
# NOTE: change filepath to match your local setup
session_table_path = "/home/matt.davis/code/ophys-mfish-dev/ophys-mfish-dev/lims_tables/ophys_sessions.json"

In [37]:
# load LIMS sessions table
def load_session_table(session_table_path):

    session_table_path = Path(session_table_path)
    df = pd.read_json(session_table_path)
    df = df.sort_values(by=['date_of_acquisition'])
    #df = df[df['date_of_acquisition'] > '2023-01-01']
    df = df[df['date_of_acquisition'] > '2018-01-01']

    df = df[df['storage_directory'].notnull()]

    print(f"Loaded {len(df)} sessions from {session_table_path}")
    return df


def get_platform_datetime_meso(session_dir):

    session_dir = Path(session_dir)

    try:
        processing_file = list(session_dir.glob('*_platform.json'))
        assert len(processing_file) == 1

        
        with open(processing_file[0]) as f:
            platform = json.load(f)

        assert "imaging_plane_groups" in platform.keys(), "imaging_plane_groups key not not in platform.json"
        platform_ds = platform["imaging_plane_groups"][0]["imaging_planes"][0]["registration"]["acquired_at"]

        # make datetime object (e.g. 2023-02-09T12:58:52.385797), ignore microseconds
        # NOTE: (Example found in table: ValueError: time data '72018-12-06T14:16:47' does not match format '%Y-%m-%dT%H:%M:%S')
        platform_dt = datetime.strptime(platform_ds.split('.')[0], '%Y-%m-%dT%H:%M:%S')
    except (AssertionError, ValueError) as e:
        platform_dt = None
        
    return platform_dt


def find_dt_match_for_data_assest(co_dt, da_id, df):
    """
    
    Parameters
    ----------
    co_dt : datetime object
        datetime object from Code Ocean data asset name.
    da_id : str
        data asset id.
    df : pandas dataframe
        dataframe of sessions table.
        
    """
    df['match'] = df['platform_datetime'].apply(lambda x: x == co_dt)
    match_session_id = df[df['match'] == True].id.values

    if len(match_session_id) == 1:
        match_session_id = match_session_id[0]
    else:
        # warn
        #print(f"WARNING: {len(match_session_id)} matches found for {da_id} {co_dt}")
        match_session_id = None

    match_dict = {'data_asset_id': da_id,
                  'session_id': match_session_id}


    return match_dict


def get_all_matched_co_and_lims_sessions(results, session_df):
    """
    Parameters
    ----------
    results : json
        json response from Code Ocean search_all_data_assets.
    session_df : pandas dataframe
        dataframe of sessions table.
        
    """
    # if results_json is dict then convert to list
    if isinstance(results, dict):
        results = results['results']
    match_list = []
    for da in results:
        match = False
        name = da['name']
        da_id = da['id']
        da_ds = name.split('_')[-2] + "-" + name.split('_')[-1]
        da_dt = datetime.strptime(da_ds, '%Y-%m-%d-%H-%M-%S')
        #mid = name.split('_')[-3]

        match_dict = find_dt_match_for_data_assest(da_dt, da_id, session_df)

        if match_dict['session_id'] is not None:
            match_dict['match'] = True

        match_list.append(match_dict)
        
    # make df
    matches = pd.DataFrame(match_list)
    return matches


In [23]:
# EXAMPLE: getting datetime object from data asset name (don't need to run this cell)

da = results[100]
da_name = da['name']
da_id = da['id']

# get datetime object from name, format: 2019-11-04_01-21-00
da_date_str = da_name.split('_')[-2] + "-" +da_name.split('_')[-1]
da_datetime = datetime.strptime(da_date_str, '%Y-%m-%d-%H-%M-%S')
da_mouse_id = da_name.split('_')[-3]

print(da_name)
print(da_datetime)
print(da_mouse_id)

multiplane-ophys_477052_2019-10-17_16-15-26
2019-10-17 16:15:26
477052


In [38]:
#data_assets_list = processed_data # don't think processed data will work
data_assets_list = raw_data

df_sessions = load_session_table(session_table_path)

df = df_sessions.copy()
df['platform_datetime'] = df['storage_directory'].apply(get_platform_datetime_meso)
df = df[df['platform_datetime'].notnull()]
print(f"Removed {len(df_sessions) - len(df)} sessions with no platform datetime")
print(f"Remaining sessions: {len(df)}")


df_match = get_all_matched_co_and_lims_sessions(data_assets_list, df)
df_match.head()
print(f"Found {len(df_match[df_match['match'] == True])} matches")

Loaded 8041 sessions from /home/matt.davis/code/ophys-mfish-dev/ophys-mfish-dev/lims_tables/ophys_sessions.json


ValueError: time data '72018-12-06T14:16:47' does not match format '%Y-%m-%dT%H:%M:%S'

In [16]:
# true matches
df_match[df_match['match'] == True]

,data_asset_id,session_id,match
26,0bd7d39f-5435-4c74-9c2c-4143e5644148,1.262863e+09,True
27,22e690d7-2c29-43a8-ad7f-366599dfae5c,1.267183e+09,True
28,53904e20-3a0b-4be4-8ac2-b0936d779410,1.307508e+09,True
29,d71eba24-36de-4707-9041-8d1f4e15139b,1.312421e+09,True
30,73117348-63cb-46b6-a1a3-e51b9bd8fe46,1.311436e+09,True
...,...,...,...
396,1db9390d-5c26-4a0d-b37a-20cbf8984b08,1.287205e+09,True
397,0c582322-1a75-45f0-8c04-12336f91bf97,1.275235e+09,True
398,48f8ea2b-e9c3-4988-bf12-107cdff7ba32,1.282665e+09,True
399,ff24db95-8407-4bba-af0c-bf255d066947,1.287478e+09,True
